In [95]:
import json
import pandas as pd

In [96]:
dataset = None
with open("sample.json", "r") as read_content: 
    dataset = (json.load(read_content))

In [97]:
print(dataset.keys())

dict_keys(['FL', 'DV'])


In [107]:
fl_data = dataset["FL"]
processed = []

def make_entry(case_id, petitioner_rep, respondent_rep, restraining_order_terms, relevant_comments):
    return {
        "case_id": case_id,
        "petitioner_rep": petitioner_rep, 
        "respondent_rep": respondent_rep,
        "restraining_order_terms": restraining_order_terms,
        "relevant_comments": relevant_comments
        }

restraining_order_terms = ["RESTRAINING","ORDER","ROAH","DV-130"]

def count_occurences(haystack, needle):
    try:
        return haystack.index(needle)
    except:
        return 0
    
for idx,case in enumerate(fl_data):
    case_object = fl_data[case]
    case_id = case_object["case"]["data"][0]["caseNumber"]
    petitioners = []
    respondents = []
    for party in case_object["event"]["data"]["caseParties"]:
        if party["type"] == "Petitioner":
            petitioners.append(party)
        if party["type"] == "Respondent":
            respondents.append(party)
    petitioners.sort(key=lambda x: len(x["fullName"]), reverse=True)
    respondents.sort(key=lambda x: len(x["fullName"]), reverse=True)
    if len(petitioners) == 0 or len(respondents) == 0:
        continue
    petitioner_full_name = petitioners[0]["fullName"]
    respondent_full_name = respondents[0]["fullName"]
    attornies = {}
    for attorney in case_object["event"]["data"]['caseAttornies']:
        representing = attorney["representing"]
        attornies[representing] = attorney
    ro_occured = False
    relevant_comments = []
    for event in case_object["event"]["data"]["caseEvents"]:
        comm = event["comment"]
        comm = comm if comm is not None else ""
        comm = comm.upper()
        occurences = [count_occurences(comm, term.upper()) for term in restraining_order_terms]
        if not ro_occured and sum(occurences) != 0:
            ro_occured = True
        if sum(occurences) != 0:
            relevant_comments.append(comm)
    respondent_attorney = respondent_full_name in attornies
    petitioner_attorney = petitioner_full_name in attornies

    #Post Processing
    relevant_comments = "\n".join(relevant_comments)


    processed.append(make_entry(case_id,petitioner_attorney,respondent_attorney,ro_occured,relevant_comments))
dataframe = pd.DataFrame(processed)
dataframe

,case_id,petitioner_rep,respondent_rep,restraining_order_terms,relevant_comments
0,23FL000002,True,True,False,
1,23FL000004,False,False,False,
2,23FL000006,False,False,False,
3,23FL000008,False,False,False,
4,23FL000009,False,False,False,
...,...,...,...,...,...
3706,23FL004245,False,False,False,
3707,23FL004246,True,False,False,
3708,23FL004250,False,False,False,
3709,23FL004252,True,False,False,


In [108]:
dataframe.to_csv("processed.csv")